In [65]:
import rasterio
import rasterio.features
import fiona
from shapely.geometry import shape, Polygon
import numpy as np
import geopandas as gdp

In [73]:
with rasterio.open('/Users/sarahauser/FHNW/4_Semester/GeoHack/GeoHack23-WildfirePerimeter/data/LWIR_QuickMosaic_16-bit_9327.tiff', 'r') as src:
    data = src.read(1)


    transform = src.transform
    crs = src.crs

    mask = (data < 35000).astype(np.uint8)
    shapes = rasterio.features.shapes(mask, transform=transform)




    all = []
    for polygon in shapes:
        a = (polygon[0])
        all.append(f'Polygon ({a["coordinates"]})')


    print(all[0])

    gdf = gdp.GeoDataFrame(geometry=all)

    print(gdf)

Polygon ([[(-122.3978768111497, 41.49975342968751), (-122.39782667978395, 41.49975342968751), (-122.39782667978395, 41.499728206250005), (-122.3978099693287, 41.499728206250005), (-122.3978099693287, 41.49965253593751), (-122.39782667978395, 41.49965253593751), (-122.39782667978395, 41.49962731250001), (-122.39789352160494, 41.49962731250001), (-122.39789352160494, 41.499677759375004), (-122.39791023206018, 41.499677759375004), (-122.39791023206018, 41.49974081796876), (-122.3978768111497, 41.49974081796876), (-122.3978768111497, 41.49975342968751)]])


TypeError: Input must be valid geometry objects: Polygon ([[(-122.3978768111497, 41.49975342968751), (-122.39782667978395, 41.49975342968751), (-122.39782667978395, 41.499728206250005), (-122.3978099693287, 41.499728206250005), (-122.3978099693287, 41.49965253593751), (-122.39782667978395, 41.49965253593751), (-122.39782667978395, 41.49962731250001), (-122.39789352160494, 41.49962731250001), (-122.39789352160494, 41.499677759375004), (-122.39791023206018, 41.499677759375004), (-122.39791023206018, 41.49974081796876), (-122.3978768111497, 41.49974081796876), (-122.3978768111497, 41.49975342968751)]])